<a href="https://colab.research.google.com/github/SavchukDI/3d_ml_mai/blob/main/3dml_lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SimSwap
Reference: https://github.com/neuralchen/SimSwap

## Prepare code

### SimSwap

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap/
!git checkout no-logo

In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy > /dev/null
!pip install googledrivedownloader > /dev/null
!pip install imageio==2.4.1 > /dev/null

### (optional) GPEN

In [ ]:
%cd /content
!git clone https://github.com/yangxy/GPEN.git

# Remove the example input
%rm -rf /content/GPEN/examples
%mkdir -p /content/GPEN/examples/imgs

In [ ]:
%pip install ninja

## Prepare models

### SimSwap

In [ ]:
%cd /content/SimSwap

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
                                    dest_path='./arcface_model/arcface_checkpoint.tar')

GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
                                    dest_path='./checkpoints.zip')

In [ ]:
!wget --no-check-certificate \
 https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w \
 -O antelope.zip

In [ ]:
!unzip ./checkpoints.zip  -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

### (optional) GPEN

In [ ]:
%cd /content/GPEN/weights
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-512.pth

## Prepare data

### Download

In [ ]:
%cd /content

!wget https://www.themoviedb.org/t/p/w600_and_h900_bestv2/ufxkgL7bALTQ1QKMRY3i3j1q25R.jpg -O adam_savage.png
!wget https://icdn4.digitaltrends.com/image/digitaltrends/obi-wan-kenobi-1300x650.jpg -O obi_wan.png

In [ ]:
input_fname = '/content/adam_savage.png'
output_fname = '/content/obi_wan.png'

### Convert to JPG

Images should not be too large, hence the (arbitrary) limitation of 1024 length.

In [ ]:
def get_new_size(img_size,
                 max_allowed_length = 1024):

  new_img_size = []
  if any(max_allowed_length < sz for sz in img_size):
    long_length = max(img_size)
    ratio = max_allowed_length / long_length
  else:
    ratio = 1.0

    new_img_size = [
                    int(ratio*sz)
                    for sz in img_size
                    ]

  return tuple(new_img_size)

In [ ]:
from PIL import Image

allow_resize = False

for fname in [input_fname, output_fname]:
  jpg_fname = fname.replace('.png', '.jpg')

  try:
    img = Image.open(fname)
  except FileNotFoundError:
    continue

  new_size = get_new_size(img.size, max_allowed_length = 1024)
  if allow_resize:
    print(f'Resizing from {img.size} to {new_size}')
    img.resize(new_size)

  print(f'Saving to {jpg_fname}')
  img.convert('RGB').save(jpg_fname)

jpg_input = input_fname.replace('.png', '.jpg')
jpg_output = output_fname.replace('.png', '.jpg')

## Run SimSwap

### Single

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/single/

!python test_wholeimage_swapsingle.py \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/single/ > /dev/null


### Multi

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/multi/

!python test_wholeimage_swapmutil.py \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/multi/ > /dev/null


## (optional) Post-process with GPEN

In [ ]:
%cp /content/output/single/result_whole_swapsingle.jpg /content/GPEN/examples/imgs/
%cp /content/output/multi/result_whole_swapmutil.jpg /content/GPEN/examples/imgs/

In [ ]:
%cd /content/GPEN
!python face_enhancement.py